<a href="https://colab.research.google.com/github/sindla97/Machine-Learning/blob/main/NER_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing library
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
#!pip install opendatasets

import opendatasets as od

od.download(
	"https://www.kaggle.com/datasets/naseralqaydeh/named-entity-recognition-ner-corpus")

In [3]:
# reading the XLSX file
file =('/content/named-entity-recognition-ner-corpus/ner.csv')
ner = pd.read_csv(file)

In [40]:


od.download(
"https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus?select=ner_dataset.csv")
data_path = "/content/entity-annotated-corpus/ner_dataset.csv"

data = pd.read_csv(data_path, encoding= 'unicode_escape')
# filling the first column that determines which sentence each word belongs to.
data.fillna(method = 'ffill', inplace = True)
data.head()

Skipping, found downloaded files in "./entity-annotated-corpus" (use force=True to force download)


,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


In [7]:
ner.head()

,Sentence #,Sentence,POS,Tag
0,Sentence: 1,Thousands of demonstrators have marched throug...,"['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...","['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '..."
1,Sentence: 2,Families of soldiers killed in the conflict jo...,"['NNS', 'IN', 'NNS', 'VBN', 'IN', 'DT', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,Sentence: 3,They marched from the Houses of Parliament to ...,"['PRP', 'VBD', 'IN', 'DT', 'NNS', 'IN', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,Sentence: 4,"Police put the number of marchers at 10,000 wh...","['NNS', 'VBD', 'DT', 'NN', 'IN', 'NNS', 'IN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,Sentence: 5,The protest comes on the eve of the annual con...,"['DT', 'NN', 'VBZ', 'IN', 'DT', 'NN', 'IN', 'D...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."


In [128]:
def tags(text):
  text=text[2:-2]
  return text.split("', '")


ner['new_tags']=ner['Tag'].apply(tags)

In [117]:
ner['Tag'][0][2:-2].split("', '")

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-geo',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-geo',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-gpe',
 'O',
 'O',
 'O',
 'O',
 'O']

In [23]:
ner['POS']=ner['POS'].apply(tags)

In [30]:
ner['Sentence_tok']=ner['Sentence'].apply(lambda x : x.split() )

In [139]:
len(ner['Sentence_tok'][0]),len(ner['POS'][0]),len(ner['new_tags'][0])

(24, 24, 24)

In [50]:
ner.shape

(47959, 6)

In [47]:
wordlist=data.Word.unique()
tags=data.Tag.unique()

In [72]:
tags

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [74]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [92]:
# cutoff reviews after 110 words
maxlen = 110

max_words = 36000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(ner['Sentence_tok'])
ner['sequences_ids'] = tokenizer.texts_to_sequences(ner['Sentence_tok'])

In [96]:
 sequences_ids=pad_sequences(ner['sequences_ids'], maxlen=maxlen, padding='post')

In [73]:
#process tag ids
tag2id={}
id2tag={}
for i,k in enumerate(tags):
  id2tag[i]=k
  tag2id[k]=i

id2tag

{0: 'O',
 1: 'B-geo',
 2: 'B-gpe',
 3: 'B-per',
 4: 'I-geo',
 5: 'B-org',
 6: 'I-org',
 7: 'B-tim',
 8: 'B-art',
 9: 'I-art',
 10: 'I-per',
 11: 'I-gpe',
 12: 'I-tim',
 13: 'B-nat',
 14: 'B-eve',
 15: 'I-eve',
 16: 'I-nat'}

In [101]:
ner

,Sentence #,Sentence,POS,Tag,new_tags,Sentence_tok,sequences_ids
0,Sentence: 1,Thousands of demonstrators have marched throug...,"['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '...","['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', ...","[Thousands, of, demonstrators, have, marched, ...","[253, 5, 966, 15, 1794, 237, 467, 6, 522, 1, 1..."
1,Sentence: 2,Families of soldiers killed in the conflict jo...,"['NNS', 'IN', 'NNS', 'VBN', 'IN', 'DT', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'...","[Families, of, soldiers, killed, in, the, conf...","[1147, 5, 148, 39, 4, 1, 505, 1088, 1, 585, 53..."
2,Sentence: 3,They marched from the Houses of Parliament to ...,"['PRP', 'VBD', 'IN', 'DT', 'NNS', 'IN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'...","[They, marched, from, the, Houses, of, Parliam...","[47, 1794, 21, 1, 2014, 5, 247, 6, 7, 1164, 4,..."
3,Sentence: 4,"Police put the number of marchers at 10,000 wh...","['NNS', 'VBD', 'DT', 'NN', 'IN', 'NNS', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'...","[Police, put, the, number, of, marchers, at, 1...","[58, 707, 1, 276, 5, 7718, 19, 2031, 188, 2613..."
4,Sentence: 5,The protest comes on the eve of the annual con...,"['DT', 'NN', 'VBZ', 'IN', 'DT', 'NN', 'I...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'...","[The, protest, comes, on, the, eve, of, the, a...","[1, 522, 539, 12, 1, 2131, 5, 1, 927, 453, 5, ..."
...,...,...,...,...,...,...,...
47954,Sentence: 47955,Indian border security forces are accusing the...,"['JJ', 'NN', 'NN', 'NNS', 'VBP', 'VBG', ...","['B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'B-gpe...","['B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', ...","[Indian, border, security, forces, are, accusi...","[365, 194, 63, 73, 29, 2153, 59, 268, 3941, 5,..."
47955,Sentence: 47956,Indian officials said no one was injured in Sa...,"['JJ', 'NNS', 'VBD', 'DT', 'NN', 'VBD', ...","['B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '...","['B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', ...","[Indian, officials, said, no, one, was, injure...","[365, 35, 17, 121, 61, 18, 521, 4, 93, 9, 444,..."
47956,Sentence: 47957,Two more landed in fields belonging to a nearb...,"['CD', 'JJR', 'VBD', 'IN', 'NNS', 'VBG', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'...","[Two, more, landed, in, fields, belonging, to,...","[40, 48, 2715, 4, 3098, 3328, 6, 7, 1186, 809, 2]"
47957,Sentence: 47958,They say not all of the rockets exploded upon ...,"['PRP', 'VBP', 'RB', 'DT', 'IN', 'DT', '...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'...","[They, say, not, all, of, the, rockets, explod...","[47, 26, 43, 152, 5, 1, 1367, 752, 1401, 1984, 2]"


In [151]:
def process_tags(lst):
  max_length=110
  processtags=[]
  for i in range(len(lst)):
    processtags.append(tag2id[lst[i]])
  processtags +=([0]*(max_length-i-1))

  return processtags

ner['new_tags2']=ner['new_tags'].apply(process_tags)


In [153]:
len(ner['new_tags2'][0])

110